### Start

In [13]:
import pandas as pd
import numpy as np

from scipy.stats import fisher_exact, mannwhitneyu

import shap
import xgboost
import inspect

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import sklearn

import matplotlib.pyplot as plt
import seaborn as sns





In [14]:
df = pd.read_csv('../data/neg_bacilli_turin_2024.txt', delimiter='\t')
print(df)
for name in df.columns:
    print(name)

    pat_id  STEMA  Achromobacter  Ochrobactrum  Aeromonas  Burkolderia  Other  \
0        1    NaN            1.0           NaN        NaN          NaN    NaN   
1        2    NaN            1.0           NaN        NaN          NaN    NaN   
2        3    NaN            1.0           NaN        NaN          NaN    NaN   
3        4    NaN            NaN           1.0        NaN          NaN    NaN   
4        5    NaN            NaN           NaN        NaN          1.0    NaN   
..     ...    ...            ...           ...        ...          ...    ...   
58      59    NaN            NaN           NaN        NaN          NaN    1.0   
59      60    1.0            NaN           NaN        NaN          NaN    NaN   
60      61    1.0            NaN           NaN        NaN          NaN    NaN   
61      62    1.0            NaN           NaN        NaN          NaN    NaN   
62      63    NaN            NaN           1.0        NaN          NaN    NaN   

    Polymicrobial  Polymicr

### Missing Data

In [15]:
print("Total missing # :", df.isna().sum().sum())
print("Total missing % :", 100* df.isna().sum().sum() / (df.shape[0]*df.shape[1]) )
df.fillna(0, inplace=True)
print("Total missing # :", df.isna().sum().sum())
print("Total missing % :", 100* df.isna().sum().sum() / (df.shape[0]*df.shape[1]) )

Total missing # : 1704
Total missing % : 33.80952380952381
Total missing # : 0
Total missing % : 0.0


### Summary Stats

In [16]:
def calculate_se_proportion(p, n):
    return np.sqrt((p * (1 - p)) / n).round(2)

In [19]:
def create_summary_table(df):
    """ Generate summary table.
    Args:
    df (DataFrame): The DataFrame for which to generate the summary table.
    Returns:
    DataFrame: The summary table."""

    # Generate summary statistics for ALL columns
    summary_table = df.describe()
    # Transpose the summary table
    summary_table = summary_table.T
    # Round all values to two decimal points
    summary_table = summary_table.round(2)
    # Create 'Mean ± Std' column
    summary_table['Mean ± Std'] = summary_table.apply(lambda row: f"{row['mean']} ± {row['std']}", axis=1)
    # Create 'Median [IQR]' column
    summary_table['Median [IQR]'] = summary_table.apply(lambda row: f"{row['50%']} [{row['25%']}-{row['75%']}]", axis=1)
    # Create 'Proportion ± SE' column
    summary_table['Proportion ± SE'] = summary_table.apply(lambda row: f"{row['mean']} ± {calculate_se_proportion(row['mean'], row['count'])}", axis=1)
    # Create 'SE_Error_Flag' column
    summary_table['SE_Error_Flag'] = summary_table['Proportion ± SE'].apply(lambda row: 1 if 'nan' in str(row) else 0)
    # Reset the index
    summary_table.reset_index(inplace=True)
    # Create 'Row_Number' column
    summary_table['Row_Number'] = summary_table.index + 1
    # Drop unnecessary columns
    summary_table.drop(['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'], axis=1, inplace=True)

    return summary_table

In [32]:
import pandas as pd

def calculate_sum_and_percentage(df):
    """ Calculate sum and percentage for each column.
    Args:
    df (DataFrame): The DataFrame for which to calculate the sum and percentage.
    Returns:
    DataFrame: A DataFrame with two columns: 'Column' for column names and 'Sum/Percentage' for their corresponding sum or percentage.
    """

    result = []
    
    total_rows = len(df)
    
    for column in df.columns:
        column_sum = df[column].sum()
        percentage = (column_sum / total_rows) * 100
        result.append([column, f"{column_sum} ({percentage:.2f}%)"])

    return pd.DataFrame(result, columns=['Column', 'Sum/Percentage'])


In [20]:
summary_table = create_summary_table(df)
summary_table.to_csv('../data/summary_table.csv')
summary_table

C:\Users\pu.0580\AppData\Local\Temp\ipykernel_14884\2192171520.py:2: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt((p * (1 - p)) / n).round(2)


,index,Mean ± Std,Median [IQR],Proportion ± SE,SE_Error_Flag,Row_Number
0,pat_id,32.0 ± 18.33,32.0 [16.5-47.5],32.0 ± nan,1,1
1,STEMA,0.49 ± 0.5,0.0 [0.0-1.0],0.49 ± 0.06,0,2
2,Achromobacter,0.16 ± 0.37,0.0 [0.0-0.0],0.16 ± 0.05,0,3
3,Ochrobactrum,0.1 ± 0.3,0.0 [0.0-0.0],0.1 ± 0.04,0,4
4,Aeromonas,0.1 ± 0.3,0.0 [0.0-0.0],0.1 ± 0.04,0,5
...,...,...,...,...,...,...
75,BSI_relapse,0.06 ± 0.25,0.0 [0.0-0.0],0.06 ± 0.03,0,76
76,Mortality_14_days,0.06 ± 0.25,0.0 [0.0-0.0],0.06 ± 0.03,0,77
77,Mortality_30_days,0.1 ± 0.3,0.0 [0.0-0.0],0.1 ± 0.04,0,78
78,Mortality_hospital,0.16 ± 0.37,0.0 [0.0-0.0],0.16 ± 0.05,0,79


In [33]:
summary_table = calculate_sum_and_percentage(df)
summary_table.to_csv('../data/summary_table.csv')
summary_table

,Column,Sum/Percentage
0,pat_id,2016 (3200.00%)
1,STEMA,31.0 (49.21%)
2,Achromobacter,10.0 (15.87%)
3,Ochrobactrum,6.0 (9.52%)
4,Aeromonas,6.0 (9.52%)
...,...,...
75,BSI_relapse,4 (6.35%)
76,Mortality_14_days,4 (6.35%)
77,Mortality_30_days,6 (9.52%)
78,Mortality_hospital,10 (15.87%)


In [34]:
summary_table

,Column,Sum/Percentage
0,pat_id,2016 (3200.00%)
1,STEMA,31.0 (49.21%)
2,Achromobacter,10.0 (15.87%)
3,Ochrobactrum,6.0 (9.52%)
4,Aeromonas,6.0 (9.52%)
...,...,...
75,BSI_relapse,4 (6.35%)
76,Mortality_14_days,4 (6.35%)
77,Mortality_30_days,6 (9.52%)
78,Mortality_hospital,10 (15.87%)


### Summary Stats + Fisher's Exat Test | Wilcoxon/Mann-Whitney Test

### Positive >2 cultures  || +50%

In [ ]:
df_cult_pos = df[df['Positive_2culture_50perc'] == 1] # 43 out of 63
summary_table = create_summary_table(df_cult_pos)
summary_table.to_csv('../data/summary_table_cult_pos.csv')

In [35]:
df_cult_pos = df[df['Positive_2culture_50perc'] == 1] # 43 out of 63
summary_table = calculate_sum_and_percentage(df_cult_pos)
summary_table.to_csv('../data/summary_table_cult_pos.csv')

In [ ]:
df_cult_neg = df[df['Positive_2culture_50perc'] == 0] # 20 out of 63
summary_table = create_summary_table(df_cult_neg)
summary_table.to_csv('../data/summary_table_cult_neg.csv')

In [36]:
df_cult_neg = df[df['Positive_2culture_50perc'] == 0] # 20 out of 63
summary_table = calculate_sum_and_percentage(df_cult_neg)
summary_table.to_csv('../data/summary_table_cult_neg.csv')

In [ ]:
exclude_cols = ['Positive_2culture_50perc', 'pat_id']
binary_results = []
numeric_results = []

In [ ]:
for col in df.columns:
    if col not in exclude_cols:
        if df[col].nunique() <= 2:
            filtered_col = df[col].dropna().astype(int)
            if filtered_col.nunique() == 2:
                contingency_table = pd.crosstab(df['Positive_2culture_50perc'], filtered_col)
                odds_ratio , p_value = fisher_exact(contingency_table)
                binary_results.append((col, odds_ratio, p_value))
            else:
                print(f"Warning: Colunm '{col}' does not exactly 2 unique values. Dropping it..." )
        else:
            group_1 = df[df['Positive_2culture_50perc'] == 1][col]
            group_2 = df[df['Positive_2culture_50perc'] == 0][col]
            _ , p_value = mannwhitneyu(group_1, group_2, alternative='two-sided')
            numeric_results.append((col, p_value))

In [ ]:
print("Numeric Variables (Wilcoxon/Mann-Whitney Test): ")
for result in numeric_results:
    print(f"{result[0]} , p-value: {result[1]}" )

In [ ]:
print("Binary Variables (Fisher's Exact Test): ")
for result in binary_results:
    print(f"{result[0]} - Odds Ratio: {result[1]}, p-value: {result[2]}" )

### In-Hospital Mortality

### Summary Stats + Fisher's Exat Test | Wilcoxon/Mann-Whitney Test

In [ ]:
df_hosp_mort_pos = df[df['Mortality_hospital'] == 1] # 10 out of 63
summary_table = create_summary_table(df_hosp_mort_pos)
summary_table.to_csv('../data/summary_table_hosp_mort_pos.csv')

In [37]:
df_hosp_mort_pos = df[df['Mortality_hospital'] == 1] # 10 out of 63
summary_table = calculate_sum_and_percentage(df_hosp_mort_pos)
summary_table.to_csv('../data/summary_table_hosp_mort_pos.csv')

In [ ]:
df_hosp_mort_neg = df[df['Mortality_hospital'] == 0] # 53 out of 63
summary_table = create_summary_table(df_hosp_mort_neg)
summary_table.to_csv('../data/summary_table_hosp_mort_neg.csv')

In [38]:
df_hosp_mort_neg = df[df['Mortality_hospital'] == 0] # 53 out of 63
summary_table = calculate_sum_and_percentage(df_hosp_mort_neg)
summary_table.to_csv('../data/summary_table_hosp_mort_neg.csv')

In [ ]:
exclude_cols = ['Mortality_hospital', 'pat_id', 'Amoxicillin_clavulanate_including', 'Metronidazole_including', 'Mortality_30_days', 'Mortality_14_days']
binary_results = []
numeric_results = []

In [ ]:
for col in df.columns:
    if col not in exclude_cols:
        if df[col].nunique() <= 2:
            filtered_col = df[col].dropna().astype(int)
            if filtered_col.nunique() == 2:
                contingency_table = pd.crosstab(df['Mortality_hospital'], filtered_col)
                odds_ratio , p_value = fisher_exact(contingency_table)
                binary_results.append((col, odds_ratio, p_value))
            else:
                print(f"Warning: Colunm '{col}' does not exactly 2 unique values. Dropping it..." )
        else:
            group_1 = df[df['Mortality_hospital'] == 1][col]
            group_2 = df[df['Mortality_hospital'] == 0][col]
            _ , p_value = mannwhitneyu(group_1, group_2, alternative='two-sided')
            numeric_results.append((col, p_value))

In [ ]:
print("Numeric Variables (Wilcoxon/Mann-Whitney Test): ")
for result in numeric_results:
    print(f"{result[0]} , p-value: {result[1]}" )

In [ ]:
print("Binary Variables (Fisher's Exact Test): ")
for result in binary_results:
    print(f"{result[0]} - Odds Ratio: {result[1]}, p-value: {result[2]}" )

### Model: XGBoost

In [ ]:
df = pd.read_csv('../data/neg_bacilli_turin_2024.txt', delimiter='\t')
df.fillna(0, inplace=True)

In [ ]:
# columns_to_drop = ['pat_id', 'Amoxicillin_clavulanate_including', 'Metronidazole_including', 'Mortality_30_days', 'Mortality_14_days']
columns_to_drop = ['pat_id', 'Amoxicillin_clavulanate_including', 'Metronidazole_including', 'Mortality_30_days', 'Mortality_14_days', 'LOS_days']
df = df.drop(columns=columns_to_drop)

In [ ]:
y = df['Mortality_hospital']
X = df.drop(columns=['Mortality_hospital'])


In [ ]:
X100 = shap.utils.sample(X, 100)

In [ ]:
df.shape

In [ ]:
y[y==1]

In [ ]:
model = xgboost.XGBClassifier(nestimators=100, max_depth=2, objective = "binary:logistic").fit(X, y)
explainer = shap.Explainer(model, X)
shap_values = explainer(X)

In [ ]:
shap_values.display_data = X.values

In [ ]:
shap.initjs()
shap.force_plot(shap_values[10])


In [ ]:
shap.force_plot(shap_values[11])


In [ ]:
shap.force_plot(shap_values[14])


In [ ]:
shap.force_plot(shap_values[41])


In [ ]:
shap.force_plot(shap_values[42])


In [ ]:
shap.force_plot(shap_values[36])


In [ ]:
shap.force_plot(shap_values[35])


In [ ]:
shap.force_plot(shap_values[2])



In [ ]:
shap.force_plot(shap_values[3])


In [ ]:
shap.force_plot(shap_values[5])


In [ ]:
shap.force_plot(shap_values[50])


In [ ]:
shap.force_plot(shap_values[60])


In [ ]:
shap.force_plot(shap_values[27])


In [ ]:
shap.plots.bar(shap_values)


In [ ]:
shap.plots.bar(shap_values.abs.max(0))


In [ ]:
shap.plots.beeswarm(shap_values)


In [ ]:
shap.plots.beeswarm(shap_values.abs, color="shap_red")

In [ ]:
shap.plots.heatmap(shap_values, max_display=10, plot_width=8)

### Principal Component Analysis

In [ ]:
df = pd.read_csv('../data/neg_bacilli_turin_2024.txt', delimiter='\t')
df.fillna(0, inplace=True)


In [ ]:
columns_to_drop = ['pat_id', 'Amoxicillin_clavulanate_including', 'Metronidazole_including', 'Mortality_14_days', 'Mortality_30_days', 'LOS_days']
df = df.drop(columns=columns_to_drop)

In [ ]:
numerical_features = df.select_dtypes(include=['float64', 'int64'])
numerical_features = numerical_features.drop(columns=['Mortality_hospital'])

binary_numerical_features = [col for col in numerical_features.columns if numerical_features[col].nunique() == 2]

non_binary_numerical_features = numerical_features.drop(columns=binary_numerical_features)

scaler = StandardScaler()
scaled_numerical_features = scaler.fit_transform(numerical_features)

In [ ]:
df[numerical_features.columns] = scaled_numerical_features

In [ ]:
df.shape

In [ ]:
numerical_features = df.drop(columns=['Mortality_hospital']).select_dtypes(include=['float64', 'int64'])
numerical_features.shape

In [ ]:
pca = PCA(n_components=0.90) 
pca.fit(numerical_features)


In [ ]:
principal_components = pca.transform(numerical_features)
principal_components_df = pd.DataFrame(data=principal_components, 
                                       columns=[f'PC{i}' for i in range(1, pca.n_components_ + 1)])


In [ ]:
loadings = pca.components_
loadings_df = pd.DataFrame(loadings, columns=numerical_features.columns)


In [ ]:
for i, component in enumerate(loadings_df.iterrows(), start=1):
    print(f"Principal Component {i} Loadings:")
    print(component)
    print("\n")

In [ ]:
loadings_df.index = loadings_df.index + 1

In [ ]:
plt.figure(figsize=(9, 16))
sns.heatmap(loadings_df.T, cmap='RdGy', annot=False, fmt=".1f", cbar=True)
plt.title('Principal Component Loadings')
plt.xlabel(None)  
plt.ylabel(None)  
plt.show()

In [ ]:
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)


plt.figure(figsize=(10, 6))
plt.plot(range(len(cumulative_variance_ratio)), cumulative_variance_ratio, marker='o', linestyle='-', color='black', markerfacecolor='none')
plt.title('Cumulative Explained Variance Ratio')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.xticks(range(len(cumulative_variance_ratio)), range(1, len(cumulative_variance_ratio) + 1))
plt.yticks(np.arange(0, 1.1, 0.1))  # Start y-axis from 0 and include increments of 0.1
plt.grid(False)
plt.show()


In [ ]:
principal_components_df

In [ ]:
y = df['Mortality_hospital']

correlation_with_y = principal_components_df.apply(lambda col: col.corr(y))


In [ ]:
correlation_with_y_sorted = correlation_with_y.abs().sort_values(ascending=False)


In [ ]:
print("Correlation with Output Variable (Absolute Values):")
print(correlation_with_y_sorted)

In [ ]:
pca_with_y = pd.concat([principal_components_df, y], axis=1)


In [ ]:
mean_loadings_by_y = pca_with_y.groupby('Mortality_hospital').mean()


In [ ]:
print("Mean Loadings by Output Variable Group:")
print(mean_loadings_by_y)

In [ ]:
mean_loadings_by_y.index = mean_loadings_by_y.index.map({0: 'No', 1: 'Yes'})


plt.figure(figsize=(25, 1))
sns.heatmap(mean_loadings_by_y, cmap='RdGy', annot=True, fmt=".1f", cbar=True)
plt.title('Mean Loadings by In-hospital Mortality Status')
plt.xlabel('Principal Components')
plt.ylabel('In-hospital Mortality')
plt.show()

In [ ]:
top_features_by_pc = {}

for pc in loadings_df.index:
    top_features_by_pc[pc] = loadings_df.loc[pc].abs().nlargest(5).index.tolist()

for pc, features in top_features_by_pc.items():
    print(f"Principal Component {pc}: {features}")
